In [44]:
!pip install -q langchain_community
!pip install -q tavily-python
!pip install -q wikipedia
!pip install -q langchain_ollama
!pip install -q bs4 duckduckgo-search
# 환경 변수 
from dotenv import load_dotenv
load_dotenv(override=True)

🕒 현재 시간: 2024-12-06 15:32:18


True

⏳ 실행 시간: 2.08초


## wikipedia retriever

In [36]:
from langchain_community.retrievers import WikipediaRetriever
retriever = WikipediaRetriever()

docs = retriever.invoke("서울")
print(docs[0].page_content[:400])


🕒 현재 시간: 2024-12-06 14:40:23
Seoul, officially Seoul Special Metropolitan City, is the capital and largest city of South Korea. The broader Seoul Capital Area, encompassing Gyeonggi Province and Incheon, emerged as the world's sixth largest metropolitan economy in 2022, trailing behind Paris, San Francisco, Los Angeles, Tokyo, and New York, and hosts more than half of South Korea's population. Although Seoul's population peak
⏳ 실행 시간: 12.54초


## 검색 - TavilySearchAPIRetriever
[tavily](https://app.tavily.com/home)

- TAVILY_API_KEY 설정(.env)

In [45]:
from langchain_community.retrievers import TavilySearchAPIRetriever

import os
import random
def get_tavily_api_key() -> str:
    return os.getenv(f"TAVILY_API_KEY_{random.randint(1, 7)}")
def get_tavily_retriever() -> TavilySearchAPIRetriever:    
    return TavilySearchAPIRetriever(k=3, api_key=get_tavily_api_key())

os.environ["TAVILY_API_KEY"] = get_tavily_api_key()

query = "서울?"
retriever = get_tavily_retriever()
retriever.invoke(query)

🕒 현재 시간: 2024-12-06 15:37:37
⏳ 실행 시간: 0.00초


## google search

# DuckDuckGo Search

https://python.langchain.com/docs/integrations/tools/ddg/




In [23]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Obama's first name?")

🕒 현재 시간: 2024-12-06 14:16:40


'Obama\'s father, Barack Obama, Sr., was a teenage goatherd in rural Kenya, won a scholarship to study in the United States, and eventually became a senior economist in the Kenyan government.Obama\'s mother, S. Ann Dunham, grew up in Kansas, Texas, and Washington state before her family settled in Honolulu.In 1960 she and Barack Sr. met in a Russian language class at the University of Hawaii ... The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] The officeholder leads the executive branch of the federal government and is the commander-in-chief of the United States Armed ... Who is the man behind the iconic name "Barack Obama"? Barack Obama\'s full name is Barack Hussein Obama II. He is an American politician and attorney who served as the 44th president of the United States fr

⏳ 실행 시간: 2.37초


In [ ]:
# google search - serper
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool, create_react_agent, AgentExecutor

google_search = GoogleSerperAPIWrapper()

tools = [
    Tool(
        name="google_search",
        func=google_search.run,
        description="Search Google for recent results"
    )
]


from langchain import hub
from langchain_ollama.llms import OllamaLLM
prompt = hub.pull("hwchase17/react")
model = OllamaLLM(model="llama3.2-vision")
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent
    ,tools=tools
    ,verbose=True
    , return_intermediate_steps=True
    )


quertion = """Tell me best Korean restaurant in Seoul.
User search tool to find the information.
To get the details, please fetch the contents from the website.
Summarize the details in 1000 words.
"""

response = agent_executor.invoke({"input": quertion})
print(response)


## langchain - chain


In [23]:
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain


model = OllamaLLM(model="llama3.2")

prompt1 = PromptTemplate.from_template("Translate {question} to English. Tell me text without addital comments")
prompt2 = PromptTemplate.from_template("Question: {english}")

chain_en = LLMChain(llm=model, prompt=prompt1, output_key="english", verbose=True)
chain2 = LLMChain(llm=model, prompt=prompt2, output_key="answer_english", verbose=True)

prompt_ko = PromptTemplate.from_template("Translate {answer_english} to Korean. Tell me text without addital comments")
chain_ko = LLMChain(llm=model, prompt=prompt_ko, verbose=True)
chain = SequentialChain(chains=[chain_en, chain2, chain_ko], input_variables=["question"], verbose=True)


chain.invoke({"question": "오늘 날씨 어때?"})



🕒 현재 시간: 2024-12-06 12:29:49


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Translate 오늘 날씨 어때? to English. Tell me text without addital comments

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Question: How is the weather today?

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Translate I'm just a language model, I don't have real-time access to current weather conditions. However, I can suggest some ways for you to find out the weather in your area.

1. Check online weather websites: You can check websites like AccuWeather, Weather.com, or the National Weather Service (NWS) for current weather conditions and forecasts.
2. Use a mobile app: There are many mobile apps available that provide real-time weather information, such as Dark Sky, Weather Underground, or The Weather Channel.
3. Tune into local news: You can watch local news or check the weather forecast on TV to g

{'question': '오늘 날씨 어때?',
 'text': '나은 나는 시간의 현재 조건에 대한 실시간 액세스를沒有합니다. 그러나ุณ은 지역의 기후 정보를 얻을 수 있는 방법을 알려드릴게요.\n\n1. 온라인 기상 사이트를 확인하십시오: AccuWeather, Weather.com, 또는 국립 기상청 (NWS)와 같은사이트를 확인하여 현재 기상 조건과 예보를 확인할 수 있습니다.\n2. 모바일 앱을 사용하십시오: 다양한 모바일 앱이 존재하고 기상 정보를 제공하는 곳이 있습니다. Dark Sky, Weather Underground, 또는 The Weather Channel와 같은 앱을 사용해 보세요.\n3. lokale 뉴스 시청하거나 확인하십시오: 지역 뉴스를 감시하거나 TV에서 기상 예보를 확인하여 현재의 기상 조건을 확인할 수 있습니다.\n\n*이 model은 텍스트 기반 AI입니다. 실시간 데이터에 대한 액세스를 가지고 있지 않으며, 현재 기상 정보를 제공할 수 없습니다.*'}

⏳ 실행 시간: 11.65초


In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain


model = OllamaLLM(model="llama3.2")

prompt_en = PromptTemplate.from_template("Translate {question} to English. Tell me text without addital comments")
prompt_q = PromptTemplate.from_template("Question: {english}")
prompt_ko = PromptTemplate.from_template("Translate {answer_english} to Korean. Tell me text without addital comments")

chain_en = prompt_en | model
chan_q = prompt_q | model
chain_ko = prompt_ko | model

chain = chain_en | chan_q | chain_ko


chain.invoke({"question": "오늘 날씨 어때?"})



In [37]:

from langchain_ollama.llms import OllamaLLM
model = OllamaLLM(model="llama3.2-vision")


from langchain.agents import create_react_agent, AgentExecutor, load_tools

HEADERS = {
    "User-Agent": "langchain/agent-search"
}

from bs4 import BeautifulSoup
import requests
def parse_html(html) -> str :
    soup = BeautifulSoup(html, "html.parser")
    text_content_with_links = soup.get_text()[:3000]
    return text_content_with_links

def fetch_website(url: str) -> str:
    if isinstance(url, list):
        url = url[0]
    print(f"Fetching : {url}")

    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)

from langchain.tools import StructuredTool, BaseTool
from langchain.prompts import PromptTemplate
web_fetch_tool = StructuredTool.from_function(func=fetch_website, name="fetch_website", description="Fetch the content of a website given its URL")

from langchain.chains import LLMChain
summarize_chain = LLMChain(llm=model, prompt=PromptTemplate.from_template(
    "Summarize the following website content: {website_content}"
) )

summarize_tool = StructuredTool.from_function(func=summarize_chain.run, name="summarize_website", description="Summarize the content of a website given its URL")

translate_korean_chain = LLMChain(llm=model, prompt=PromptTemplate.from_template(
    "Translate the following text to Korean: {text}"
))
translate_korean_tool = StructuredTool.from_function(func=translate_korean_chain.run, name="translate_text", description="Translate the text to Korean")

# from langchain.tools import DuckDuckGoSearchResults
# ddg_search = DuckDuckGoSearchResults()
# tools = [ddg_search, translate_korean_tool]

from langchain_community.retrievers import WikipediaRetriever
retriever = WikipediaRetriever()

# docs = retriever.invoke("서울")
# print(docs[0].page_content[:400])

tools = [retriever.as_tool()]


# from langchain import hub
# prompt = hub.pull("hwchase17/react")
prompt = PromptTemplate.from_template(
    """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. Translate to Korean.

Begin!

Question: {input}
Thought:{agent_scratchpad}"""
)

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent
    ,tools=tools
    ,verbose=True
    ,return_intermediate_steps=True
    )


quertion = """오늘 인천 날씨 알려줘."""
response = agent_executor.invoke({"input": quertion})
print(response)


🕒 현재 시간: 2024-12-06 14:41:03


> Entering new AgentExecutor chain...
Thought: 먼저 인천의 날씨를 확인할 수 있는 한국어 위키백과 문서가 필요할 것 같다.

Action: WikipediaRetriever
Action Input: {'title': '인천광역시 날씨', 'type': 'string'}
[]It seems like the WikipediaRetriever function didn't return any results. Let's try again with a different title.

Action: WikipediaRetriever
Action Input: {'title': '인천광역시', 'type': 'string'}
[]Question: 오늘 인천 날씨 알려줘.
Thought: 먼저 인천의 날씨를 확인할 수 있는 한국어 위키백과 문서가 필요할 것 같다.

Action: WikipediaRetriever
Action Input: {'title': '인천광역시 날씨', 'type': 'string'}
[]It seems like the function is not returning any results. Let's try a different approach.

Thought: Incheon의 날씨 관련 문서가 있나?

Action: WikipediaRetriever
Action Input: {'title': '인천시', 'type': 'string'}
[]Question: 오늘 인천 날씨 알려줘.
Thought: 먼저 인천의 날씨를 확인할 수 있는 한국어 위키백과 문서가 필요할 것 같다.

Action: WikipediaRetriever
Action Input: {'title': '인천광역시 날씨', 'type': 'string'}
[]Question: 오늘 인천 날씨 알려줘.
Thought: 먼저 인천의 날씨를 확인할 수 있는 한국어 위키백과 문서가 필요할 것 같다.

Act